In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

from path import Path
import pandas as pd
from IPython.display import clear_output
import tensorflow as tf

In [2]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [17]:
train_path = tf.keras.utils.get_file('iris_training.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv')
test_path = tf.keras.utils.get_file('iris_test.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv')

clear_output()

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

print(train.head())

train_y = train.pop('Species')
test_y = test.pop('Species')

   SepalLength  SepalWidth  PetalLength  PetalWidth  Species
0          6.4         2.8          5.6         2.2        2
1          5.0         2.3          3.3         1.0        1
2          4.9         2.5          4.5         1.7        2
3          4.9         3.1          1.5         0.1        0
4          5.7         3.8          1.7         0.3        0


In [18]:
train.shape

(120, 4)

In [19]:
def input_fn(features, labels, training=True, batch_size=256):
    ds = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if training:
        ds = ds.shuffle(1000).repeat()
    return ds.batch(batch_size)

In [20]:
feature_columns = []
for key in train.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))
print(feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [22]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively
    hidden_units=[30, 10],
    # The model must choose between 3 classes
    n_classes=3
)
clear_output()

In [24]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)
clear_output()

In [25]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-07-22T09:07:53
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\SC-23431\AppData\Local\Temp\tmp_ph1m_i3\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.30700s
INFO:tensorflow:Finished evaluation at 2021-07-22-09:07:53
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.8666667, average_loss = 0.42855814, global_step = 10000, loss = 0.42855814
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: C:\Users\SC-23431\AppData\Local\Temp\tmp_ph1m_i3\model.ckpt-10000

Test set accuracy: 0.867



In [33]:
def input_fn(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(features).batch(batch_size)

predict = {}

print('Please type numeric values as prompted')
for feature in train.keys():
    valid = False
    while not valid:
        val = input(feature + ': ')
        try:
            predict[feature] = [float(val)]
            valid = True
        except ValueError:
            pass

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    for class_id in pred_dict['class_ids']:
        probability = pred_dict['probabilities'][class_id]
        print('Prediction is: "{}" ({:.1f}%)'.format(SPECIES[class_id], 100 * probability))

Please type numeric values as prompted
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\SC-23431\AppData\Local\Temp\tmp_ph1m_i3\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is: "Virginica" (62.7%
